In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input, InputLayer
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils
import h5py

Using TensorFlow backend.


In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)  # set this TensorFlow session as the default session for Keras  # set this TensorFlow session as the default session for Keras

In [3]:
os.chdir(r'D:\ffmpeg\directorios\experiments\npimages')
files = glob.glob(r'*[0-9].npy')
files.sort(key = len)
print(files)
scorefiles = [i for i in files if 'score' in i]
videofiles = [i for i in files if 'score' not in i]

['1.npy', '2.npy', '3.npy', '4.npy', '5.npy', '6.npy', '7.npy', '8.npy', '9.npy', '10.npy', '11.npy', '12.npy', '13.npy', '14.npy', '15.npy', '16.npy', '17.npy', '18.npy', '19.npy', '20.npy', '21.npy', '22.npy', '23.npy', '24.npy', '25.npy', '26.npy', '27.npy', '28.npy', '29.npy', '30.npy', '31.npy', '32.npy', '33.npy', '34.npy', '35.npy', '36.npy', '37.npy', '38.npy', '39.npy', '40.npy', '41.npy', '42.npy', '43.npy', '44.npy', '45.npy', '46.npy', '47.npy', '48.npy', '49.npy', '50.npy', '51.npy', '52.npy', '53.npy', '54.npy', '55.npy', '56.npy', '57.npy', '58.npy', '59.npy', '60.npy', '61.npy', '62.npy', '63.npy', '64.npy', '65.npy', '66.npy', '67.npy', '68.npy', '69.npy', '70.npy', '71.npy', '72.npy', '73.npy', '74.npy', '75.npy', '76.npy', '77.npy', '78.npy', '79.npy', '80.npy', '81.npy', '82.npy', '83.npy', '84.npy', '85.npy', '86.npy', '87.npy', '88.npy', '89.npy', '90.npy', '91.npy', '92.npy', '93.npy', '94.npy', '95.npy', '96.npy', '97.npy', '98.npy', '99.npy', '100.npy', '101.np

In [4]:
def myGenerator():
    while True:
        ls = random.sample(range(1, 800), 2)
        alldata_x = []
        alldata_y = []
        for i in ls:
            f = files[i]
            s = f[:-4]+'_score.npy'
            #print(f,s)
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
            #print(alldata_x)
            #print(alldata_y)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y
x = myGenerator()
xtrain, ytrain = next(x)
print('xtrain shape:',xtrain.shape)
print('ytrain shape:',ytrain.shape)

(2, 50, 240, 320, 3) (2, 5)
xtrain shape: (2, 50, 240, 320, 3)
ytrain shape: (2, 5)


In [5]:
print(len(videofiles))
for f in videofiles:
    print(np.load(f).shape)
    break

800
(50, 240, 320, 3)


In [19]:
height = 240
width = 320
input_shape=(10, height, width, 3)

In [28]:
def mySegNet(input_shape):
    base_model  = VGG16(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer = optimizers.Adam(lr=0.0001), loss='categorical_crossentropy')
    print(model.summary())
    return model 
mySegNet(input_shape)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 2, 512)            14714688  
_________________________________________________________________
time_distributed_18 (TimeDis (None, 2, 512)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_15 (Dense)             (None, 100)               51300     
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 505       
Total params: 16,865,693
Trainable params: 16,865,693
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer = optimizers.Adam(lr=0.00001), loss='categorical_crossentropy')
    print(model.summary())
    return model 
mySegNet(input_shape)

In [ ]:
m = mySegNet(input_shape)
m.layers.pop()
m.outputs = [m.layers[-1].output]
m.layers[-1].outbound_nodes = []
print(m.summary())

In [ ]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=False,
                   steps_per_epoch=20, epochs=20)
path = 'D:/ffmpeg/directorios/experiments/'
model.save(path+'model1.h5')
model.save_weights(path+'model_weights1.h5')

In [ ]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=False,
                   steps_per_epoch=1, epochs=1)
path = 'D:/ffmpeg/directorios/experiments/'
model.save(path+'model1.h5')
model.save_weights(path+'model_weights1.h5')